---
title: "Phase Diagram"
author: "Tu"
date: "2025-09-04"
categories: [macro, code, visualization]
execute:
  enabled: false
---

Phase diagram 是用来可视化系统随时间演化的绝佳手段，这里我们通过可视化经济学经典的RCK模型来说明 phase diagram 的含义。

一个典型的连续时间新古典增长模型最终的一般均衡结果通常可以由下述的微分方程组描述：

$$
\begin{align*}
    \dot{k}(t) &= f(k(t)) - (n + g)k(t) - c(t) \\
    \dot{c}(t) &= c(t)(f'(k(t)) - \rho - g) \\[1ex]
    \text{with} \enspace &k(0) = k_0 > 0, \enspace \lim_{t \to \infty} k(t) \exp(-\int_{0}^{t} (f'(k(s)) - n - g) \, ds) = 0 \\
    &k(t) \ge 0, \enspace c(t) \ge 0
\end{align*}
$$ {#eq-01}

这个两变量的一阶微分方程组实际上定义了一个从 $\mathbb{R}_+^2$ 到 $\mathbb{R}^2$ 的映射，即给定任意的 $(k, c)\in\mathbb{R}_+^2$， 对应于一个新的向量 $(\dot{k},\dot{c})\in\mathbb{R}^2$。 从几何的角度来看，这正是一个向量场：想象每一个二维平面正象限的点上都附着了一个新的向量，这个向量正好是 $k(t)$ 和 $c(t)$ 对时间的导数，因此这个向量的方向决定了这个点当前运动的方向，这个向量的大小决定了这个点运动速度的大小。

我们使用[`quiver()`](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.quiver.html#matplotlib.axes.Axes.quiver)将这一向量场在 $(k, c)$ 所在的二维平面上可视化出来：

In [ ]:
# %%
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from numpy.linalg import eig
from matplotlib.animation import FuncAnimation
import warnings

# 忽略 Matplotlib 可能产生的一些用户警告
warnings.filterwarnings("ignore", category=UserWarning)

# 确保 Pillow 库已安装
try:
    import PIL
except ImportError:
    print("请安装 Pillow 库: pip install Pillow")
    exit()

# 提高 Agg 后端的内存限制，以处理复杂的动画帧
plt.rcParams['agg.path.chunksize'] = 10000

# %%
# 定义微分方程组
def rck(state, t, alpha, rho, n, g):
    k, c = state

    # 添加安全检查，防止 k 变为负数导致数学错误
    if k <= 0:
        return [0, 0] # 如果 k 非正，则停止运动
    
    dkdt = k**alpha - c - (n + g)*k
    dcdt = c*(alpha*k**(alpha-1) - rho - g)
    return [dkdt, dcdt]

# %%
# 定义参数
alpha, rho, n, g = 0.3, 0.03, 0.02, 0.02

# %%
# 计算稳态
k_right = (1/(n+g))**(1/(1-alpha))
k_gloden = (alpha/(n+g))**(1/(1-alpha))
c_gloden = k_gloden**alpha - (n + g)*k_gloden
k_star = (alpha / (rho + g))**(1 / (1 - alpha))
c_star = k_star**alpha - (n + g) * k_star

# %%
# 计算 Jacobian 矩阵及其特征值和特征向量
def rck_jacobian(k, c, alpha, rho, n, g):
    # 计算雅可比矩阵的四个元素
    dk_dot_dk = alpha * k**(alpha - 1) - (n + g)
    dk_dot_dc = -1.0
    
    dc_dot_dk = c * (alpha * (alpha - 1) * k**(alpha - 2))
    # 在稳态处，(alpha*k**(alpha-1) - rho - delta - theta*g) = 0
    # 所以 dc_dot/dc = 0，这是 RCK 线性化的一个特点
    dc_dot_dc = 0.0 
    
    return np.array([[dk_dot_dk, dk_dot_dc], [dc_dot_dk, dc_dot_dc]])

#在稳态点计算雅可比矩阵
J = rck_jacobian(k_star, c_star, alpha, rho, n, g)

# 计算特征值和特征向量
eigenvalues, eigenvectors = eig(J)

# 找到稳定特征向量 (与负特征值对应的那个)
stable_eigenvector = eigenvectors[:, np.argmin(eigenvalues)]

# %%
# 计算 saddle path
saddle_point = [k_star, c_star]
t_backward = np.linspace(0, -100, 600)
path_saddle_up = odeint(rck, saddle_point + 0.1 * stable_eigenvector, t_backward, args=(alpha, rho, n, g))
path_saddle_down = odeint(rck, saddle_point - 0.1 * stable_eigenvector, t_backward, args=(alpha, rho, n, g))
path_saddle_up = np.flipud(path_saddle_up)
path_saddle_down = np.flipud(path_saddle_down)

# %%
# 计算背景向量场
k_grid = np.linspace(0.01, k_right*1.05, 20)
c_grid = np.linspace(0.01, c_gloden*1.8, 20)
K, C = np.meshgrid(k_grid, c_grid)
u, v = np.zeros(K.shape), np.zeros(C.shape)
for i in range(K.shape[0]):
    for j in range(K.shape[1]):
        dkdt, dcdt = rck([K[i, j], C[i, j]], 0, alpha, rho, n, g)
        u[i, j] = dkdt
        v[i, j] = dcdt

# 初始化图像，并设置背景为透明
fig, ax = plt.subplots(figsize=(6, 5))
fig.tight_layout()  # 调整边距
fig.patch.set_alpha(0.0)
ax.patch.set_alpha(0.0)
ax.set_xlim(0, k_right*1.05)
ax.set_ylim(0, c_gloden*1.8)
ax.set_xlabel('k')
ax.set_ylabel('c')

# %%
# 绘制 loci：dkdt=0，dcdt=0
x = np.linspace(0, k_right*1.05, 200)
y = x**alpha - (n + g)*x
ax.plot(x, y, 
        color='black',
        linestyle='-', 
        linewidth=1.5)
ax.axvline(x = k_star, color= 'black',linestyle='-', linewidth=1.5)

# %%
# 绘制 saddle path
ax.plot(path_saddle_up[:,0], path_saddle_up[:,1],
        color='black',
        linestyle='-', 
        linewidth=1.8)

ax.plot(path_saddle_down[:,0], path_saddle_down[:,1],
        color='black',
        linestyle='-', 
        linewidth=1.8)

# 绘制向量场
quiv = ax.quiver(K, C, u, v, 
                     color='gray', 
                     alpha=0.6,
                     angles='xy',
                     scale_units='xy',
                     scale=0.5,
                     width=0.005)

plt.savefig('rck_quiver.png', bbox_inches='tight')
plt.close(fig)

![](rck_quiver.png)

想象我们向这个二维平面中抛入一个小球，图中的每一个箭头都指示了如果小球到达该位置时，这个小球接下来将会如何运动。当小球按照箭头连续地运动起来，它的的运动轨迹便构成了流线，我们使用[`streamplot()`](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.streamplot.html#matplotlib.axes.Axes.streamplot)将这些流线可视化：

In [ ]:
# %%
# 计算背景向量场
k_grid = np.linspace(0.01, k_right*1.05, 100)
c_grid = np.linspace(0.01, c_gloden*1.8, 100)
K, C = np.meshgrid(k_grid, c_grid)
u, v = np.zeros(K.shape), np.zeros(C.shape)
for i in range(K.shape[0]):
    for j in range(K.shape[1]):
        dkdt, dcdt = rck([K[i, j], C[i, j]], 0, alpha, rho, n, g)
        u[i, j] = dkdt
        v[i, j] = dcdt

# 初始化图像，并设置背景为透明
fig, ax = plt.subplots(figsize=(6, 5))
fig.tight_layout()  # 调整边距
fig.patch.set_alpha(0.0)
ax.patch.set_alpha(0.0)
ax.set_xlim(0, k_right*1.05)
ax.set_ylim(0, c_gloden*1.8)
ax.set_xlabel('k')
ax.set_ylabel('c')

# %%
# 绘制 loci：dkdt=0，dcdt=0
x = np.linspace(0, k_right*1.05, 200)
y = x**alpha - (n + g)*x
ax.plot(x, y, 
        color='black',
        linestyle='-', 
        linewidth=2)
ax.axvline(x = k_star, color= 'black',linestyle='-', linewidth=1.5)

# %%
# 绘制 saddle path
ax.plot(path_saddle_up[:,0], path_saddle_up[:,1],
        color='black',
        linestyle='-', 
        linewidth=1.8)

ax.plot(path_saddle_down[:,0], path_saddle_down[:,1],
        color='black',
        linestyle='-', 
        linewidth=1.8)

# %%
# 绘制流线图
strm = ax.streamplot(K, C, u, v, 
                     color=(0.4, 0.4, 0.4, 0.4), 
                     linewidth=1.2, 
                     density=1.2,)

plt.savefig('rck_stream.png')
plt.close(fig)

![](rck_stream.png)

直观感受，此时的二维平面像是充满了流动的液体，其流动的规则由我们的微分方程组描述。进一步，想象我们向其中抛入一堆小球，让我们用[`FuncAnimation()`](https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.FuncAnimation.html#matplotlib.animation.FuncAnimation)将这些小球的运动可视化：

In [ ]:
# %%
# 计算粒子运动轨迹

# 在 k 轴上从接近 0 到 k_max 创建 4 个均匀分布的点
k_ticks = np.linspace(5, 60, 4)

# 在 c 轴上从接近 0 到 c_max 创建 4 个均匀分布的点
c_ticks = np.linspace(0.25, 2.5, 4)

# K 和 C 都将是 4x4 的矩阵
K_grid, C_grid = np.meshgrid(k_ticks, c_ticks)

# K_grid.flatten() 会将 4x4 的 K 矩阵变成一个 16x1 的向量
# C_grid.flatten() 会将 4x4 的 C 矩阵变成一个 16x1 的向量
k_initial = K_grid.flatten()
c_initial = C_grid.flatten()

# 使用 column_stack 将它们合并成一个 16x2 的矩阵
initial_states = np.column_stack([k_initial, c_initial])

t = np.linspace(0, 50, 300)
paths = [odeint(rck, state, t, args=(alpha, rho, n, g)) for state in initial_states]

# %%
paths.append(path_saddle_up[100:400, :])
paths.append(path_saddle_down[150:450, :])

# %%
# 创建动画元素和更新函数
particles, = ax.plot([], [], "ro", ms=4)

def update(frame):
    x_data = [p[frame, 0] for p in paths]
    y_data = [p[frame, 1] for p in paths]
    particles.set_data(x_data, y_data)
    return particles,

# 创建动画对象
ani = FuncAnimation(fig, update, frames=len(t), interval=20, blit=True)

# 保存动画为 APNG 文件
ani.save(
    "rck_animation.png",
    writer='pillow',
    fps=50,
    savefig_kwargs={'transparent': True, 'facecolor': 'none'}
)

![](rck_animation.png)

目前为止我们只用到了我们完整的均衡描述[(1)](#eq-01)的前两个运动方程，即任意给定小球的初始位置 $(k(0),c(0))\in\mathbb{R}_+^2$，小球会如何运动下去。换言之，图中任何一个小球的运动轨迹都是这个微分方程组的解，对于这个两变量的一阶微分方程组，我们需要给定两个初始条件才能唯一地确定一个解。

但注意到，我们RCK模型的解的描述中似乎只有一个初始条件 $k(0)=k_0 > 0$，另一个初始条件是什么？答案是 $\lim_{t \to \infty} k(t) \exp(-\int_{0}^{t} (f'(k(s)) - n - g) \, ds) = 0$，虽然将之称为边界条件或者终点条件更为合适，但它确实也是某种意义上的“初始条件”。事实上，对于任意给定的初始资本 $k(0)=k_0 > 0$ ，这个边界条件反过来唯一地确定了初始消费 $c(0)$ 的取值，这唯一的对应关系构成了从状态变量 $k$ 到控制变量 $c$ 的对应关系 $c=g(k)$，称之为 policy function，体现在图中便是我们的 saddle path，这背后是家庭的最优化行为。

从图中可以很清晰地看到，任何在 saddle path 上方的小球最终会运动到左上角，任何在 saddle path 下方的小球最终会运动到右下角，他们看起来都是不合理的解，只有刚好在 saddle path 上的小球最终会沿着这条唯一确定的最优路径抵达稳态，值得注意的是即使是初始值非常微小的偏差，最终也会发散。

虽然左上角和右下角直觉上都是不合理的区域，但我们最终一般均衡的描述[(1)](#eq-01)应当自包含了唯一正确的解，如何严格排除其他不合理的解？

可以验证：

- 右下角最终会违反 $\lim_{t \to \infty} k(t) \exp(-\int_{0}^{t} (f'(k(s)) - n - g) \, ds) = 0$

- 左上角最终会使得 $k(t)$ 为负，违反了物理约束 $k(t) \ge 0$